In [1]:
from tools import *

/Users/akhilesh/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
##### Don't need to run - one time task of reading the dump and creating a corpus
rerun=True
if rerun:
    loc ="/Users/akhilesh/Desktop/Presentations/"
    universal_doc=doc_creator(loc) 
    corpus= cleanit(list(universal_doc.values()))
    vectorizer=vectorizer_creation(corpus)
    slide_names=list(universal_doc.keys())
    corp_vector=list(vector(corpus,vectorizer))
    f = open('/Users/akhilesh/Desktop/Presentations/qER for pharma (tPA).pptx', "rb")
    standard=Presentation(f)
    pickle.dump(universal_doc, open( "/Users/akhilesh/Desktop/universal_doc.pickle", "wb" ))
    pickle.dump(corpus, open( "/Users/akhilesh/Desktop/corpus.pickle", "wb" ))
    
    
path11='/Users/akhilesh/Desktop/'#input('Please enter the directory of the folder with pickle files:')
universal_doc = pickle.load( open( path11+"universal_doc.pickle", "rb" ))
corpus=pickle.load( open( path11+"corpus.pickle", "rb" ))
vectorizer=vectorizer_creation(corpus)
slide_names=list(universal_doc.keys())
corp_vector=list(vector(corpus,vectorizer))
tf_vectorizer=tf_vectorizer_creation(corpus)
corp_tf_vector=tf_vector(corpus,tf_vectorizer)

def tf_notsameslide(slide,final_vector):
    if len(final_vector)==0:
        return True
    sim=[]
    for i in range(len(final_vector)):
        sim.append(1-cosine(final_vector[i],slide))
    #print(sim)
    ind=sim.index(max(sim))
    if max(sim)>0.95:
        #print("in",'\n')
        if np.count_nonzero(final_vector[ind])>=np.count_nonzero(slide):
            return False
        else:
            del final_vectors[ind]
            del final_result[ind]
            return True
    else:
        return True


In [3]:
%%time
#keyword matching to the dump of slides - returns list of ppts and the slides numbers with hits
key=input("Please enter the keywords : ")
final_result=[]
final_vectors=[]
key_vector=list(vector(cleanit([key])*len(corpus),vectorizer))
result=np.einsum('ij, ij->i', key_vector, corp_vector)#np.multiply(key_vector, corp_vector).sum(1)
similarity=result/sum(key_vector[0])
for i in range(len(result)):
    if similarity[i]>=0.75:
        #print(i)
        #print(len(final_vectors))
        if tf_notsameslide(corp_tf_vector[i],final_vectors):
            final_result.append(slide_names[i])
            final_vectors.append(corp_tf_vector[i])
print('\n'.join(final_result))
#creates ppts and saves them to a location provided by the user
from pptx import Presentation
source_dir='/Users/akhilesh/Desktop/Presentations/'
target_dir=source_dir#input("Please enter the directory where the ppt needs to be saved : ")
searchresult1=Presentation()
searchresult2=Presentation()
print('Generating PPT',end='')
for i in range(len(final_result)):
    source=source_dir+' '.join(final_result[i].split()[:-2])
    f = open(source, "rb")
    prs=Presentation(f)
    slide=final_result[i].split()[-1]
    copy_slide_from_external_prs(prs,int(slide), searchresult1,searchresult2)
    print('.',end='')
searchresult1.save(target_dir+'search_result1.pptx')
if len(searchresult2.slides)>0:
    searchresult2.save(target_dir+'search_result2.pptx')

Please enter the keywords : world
Qure intro Qualcomm v2.pptx Slide: 10
qER Intro v2.1.pptx Slide: 27
qXR AZ Portfolio v2.pptx Slide: 10
qXR AZ Portfolio v2.pptx Slide: 16
qXR AZ Portfolio v2.pptx Slide: 21
qQuant Intro.pptx Slide: 7
qXR COVID Technical.pptx Slide: 18
AZ_LatAm_12th Feb.pptx Slide: 5
search_result1.pptx Slide: 9
search_result1.pptx Slide: 10
search_result1.pptx Slide: 11
Generating PPT.........

KeyError: 'rId30'

In [ ]:
def get_slide(query):
    def tf_notsameslide(slide,final_vector):
        if len(final_vector)==0:
            return True
        sim=[]
        for i in range(len(final_vector)):
            sim.append(1-cosine(final_vector[i],slide))
        #print(sim)
        ind=sim.index(max(sim))
        if max(sim)>0.95:
            #print("in",'\n')
            if np.count_nonzero(final_vector[ind])>=np.count_nonzero(slide):
                return False
            else:
                del final_vectors[ind]
                del final_result[ind]
                return True
        else:
            return True
    key=query
    final_result=[]
    final_vectors=[]
    key_vector=list(vector(cleanit([key])*len(corpus),vectorizer))
    result=np.einsum('ij, ij->i', key_vector, corp_vector)#np.multiply(key_vector, corp_vector).sum(1)
    similarity=result/sum(key_vector[0])
    for i in range(len(result)):
        if similarity[i]>=0.75:
            #print(i)
            #print(len(final_vectors))
            if tf_notsameslide(corp_tf_vector[i],final_vectors):
                final_result.append(slide_names[i])
                final_vectors.append(corp_tf_vector[i])
    print('\n'.join(final_result))
    #creates ppts and saves them to a location provided by the user
    from pptx import Presentation
    source_dir='/Users/akhilesh/Desktop/Presentations/'
    target_dir=source_dir#input("Please enter the directory where the ppt needs to be saved : ")
    searchresult1=Presentation()
    searchresult2=Presentation()
    print('Generating PPT',end='')
    for i in range(len(final_result)):
        source=source_dir+' '.join(final_result[i].split()[:-2])
        f = open(source, "rb")
        prs=Presentation(f)
        slide=final_result[i].split()[-1]
        copy_slide_from_external_prs(prs,int(slide), searchresult1,searchresult2)
        print('.',end='')
    searchresult1.save(target_dir+'search_result1.pptx')
    if len(searchresult2.slides)>0:
        searchresult2.save(target_dir+'search_result2.pptx')
    return('Your PPTs are saved at '+target_dir)

In [ ]:
get_slide('qer validation')

In [ ]:
from flask import Flask
from flask_restplus import Api, Resource, reqparse
from werkzeug.utils import cached_property
app = Flask(__name__)
api = Api(app)
parser = reqparse.RequestParser()
parser.add_argument('keywords', help='Specify the keywords:')
@api.route('/qppt/')
class slide_finder(Resource):
    @api.doc(parser=parser)
    def get(self):        
        args = parser.parse_args()
        keywords = args['keywords']
        output=get_slide(keywords)
        return output
    
if __name__ == '__main__':
    app.run()